In [1]:
import os 
import sys 
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_google_genai  import GoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq

In [2]:
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
groq_api_key=os.getenv('GROQ_API_KEY')

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="llama-3.3-70b-versatile")

In [4]:
loader = PyPDFLoader("C:\\KaiBot\\Human_Segmentation_Research.pdf")

In [5]:
doc = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(doc[:20])  # splitting

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [8]:
from langchain_community.vectorstores import FAISS

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(final_documents, embeddings)


C:\Users\HP\AppData\Local\Temp\ipykernel_20904\2107674846.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\MultimodalRAg\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#vectorstore = FAISS.from_documents(final_documents,embeddings)

In [12]:
prompt=ChatPromptTemplate.from_template(
"""
You are an AI research assistant with expertise in analyzing academic papers. 
Answer the user's question based only on the provided context.
If the answer is not found in the context, state that clearly.
Context: {context}
Questions:{input}

"""
)

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [14]:
retriver=vectorstore.as_retriever()


In [15]:
import time 
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vectorstore.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
start=time.process_time()
response=retrieval_chain.invoke({'input':"Explain the summary of the paper in detail.",
                                 'context':" ".join([doc.page_content for doc in final_documents[:3]])})
print("Response time :",time.process_time()-start)
#print(response['answer'])
if isinstance(response['answer'], list):  
    print(response['answer'][0])  # take the first response
else:
    print(response['answer'])

Response time : 0.171875
Based on the provided context, it appears that the paper discusses a comparison of different methods for semantic segmentation, a task in computer vision where the goal is to assign a label to each pixel in an image. 

The paper computes the means of various metrics, including validation loss, training loss, validation IoU (Intersection over Union), and training IoU, for different encoders and optimizers. These means are compiled into comparative tables, allowing for clear benchmarking between methods. Additionally, graphical representations, such as bar plots, are created to visualize these trends.

The paper also references various other research papers and techniques, including U-Net, SegNet, and EfficientNet, which are all deep learning architectures used for semantic segmentation. The context also mentions statistical analysis and performance evaluation of deep learning-based segmentation methods.

However, the context does not provide a detailed summary o

In [17]:


# The .stream() method returns a generator that yields dictionary chunks.
# We iterate through the generator to get the response as it's created.
for chunk in retrieval_chain.stream({'input': "Summarize the methodology."}):
    # The actual generated text is usually in the 'answer' key of the chunk.
    if "answer" in chunk:
        # Print the chunk of the answer without a newline, and flush the output
        # to ensure it appears immediately in the console.
        print(chunk['answer'], end="", flush=True)

#   print("\n\nResponse time :", time.process_time() - start)

The methodology involves computing the means of various performance metrics, compiling them into comparative tables, and creating graphical representations (such as bar plots) to visualize trends. Specifically, the study reports six statistical measures (mean, standard deviation, minimum, maximum, skewness, and kurtosis) for both loss and Intersection over Union (IoU) on training and validation sets for different optimization algorithms (e.g., Adam, SGD, RAdam) and encoder models (e.g., U-Net, EfficientNet, ResNet).